In [5]:
%pip install -q PyQt5
%pip install -q Pillow

Note: you may need to restart the kernel to use updated packages.

  Using cached pillow-10.4.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
Using cached pillow-10.4.0-cp311-cp311-win_amd64.whl (2.6 MB)


## Json → Yolo 데이터 구조 변환

In [1]:
import json
import os
from PIL import Image
import shutil
import yaml

# 고양이 포즈를 클래스 ID로 매핑
CAT_POSE_CLASSES = {
    'CAT_ARCH': 0, 'CAT_ARMSTRETCH': 1, 'CAT_FOOTPUSH': 2, 'CAT_GETDOWN': 3, 'CAT_GROOMING': 4,
    'CAT_HEADING': 5, 'CAT_LAYDOWN': 6, 'CAT_LYING': 7, 'CAT_ROLL': 8, 'CAT_SITDOWN': 9,
    'CAT_TAILING': 10, 'CAT_WALKRUN': 11
}

# 개 포즈를 클래스 ID로 매핑
DOG_POSE_CLASSES = {
    'DOG_BODYLOWER': 12, 'DOG_BODYSCRATCH': 13, 'DOG_BODYSHAKE': 14, 'DOG_FEETUP': 15, 'DOG_FOOTUP': 16,
    'DOG_HEADING': 17, 'DOG_LYING': 18, 'DOG_MOUNTING': 19, 'DOG_SIT': 20, 'DOG_TAILING': 21,
    'DOG_TAILLOW': 22, 'DOG_TURN': 23, 'DOG_WALKRUN': 24
}

ALL_POSE_CLASSES = {**CAT_POSE_CLASSES, **DOG_POSE_CLASSES}

def convert_bbox(img_width, img_height, bbox):
    x = bbox['x']
    y = bbox['y']
    width = bbox['width']
    height = bbox['height']
    
    x_center = (x + width / 2) / img_width
    y_center = (y + height / 2) / img_height
    width = width / img_width
    height = height / img_height
    
    return x_center, y_center, width, height

def convert_keypoint(img_width, img_height, keypoint):
    x = float(keypoint['x']) / img_width
    y = float(keypoint['y']) / img_height
    return x, y

def convert_json_to_yolo(json_file, img_folder, output_img_dir, output_label_dir, output_meta_dir, class_id):
    try:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except UnicodeDecodeError:
        with open(json_file, 'r', encoding='cp949') as f:
            data = json.load(f)
    
    os.makedirs(output_img_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)
    os.makedirs(output_meta_dir, exist_ok=True)
    
    # 비디오 레벨 메타데이터 추출 및 저장
    video_metadata = data.get('metadata', {})
    video_id = os.path.splitext(os.path.basename(json_file))[0]
    meta_filename = f"{video_id}_meta.json"
    meta_path = os.path.join(output_meta_dir, meta_filename)
    with open(meta_path, 'w', encoding='utf-8') as f:
        json.dump(video_metadata, f, ensure_ascii=False, indent=2)
    
    missing_images = []
    
    for annotation in data['annotations']:
        frame_number = annotation['frame_number']
        timestamp = annotation['timestamp']
        
        img_filename = f"frame_{frame_number}_timestamp_{timestamp}.jpg"
        img_path = os.path.join(img_folder, img_filename)
        
        if not os.path.exists(img_path):
            missing_images.append(img_filename)
            continue
        
        try:
            with Image.open(img_path) as img:
                img_width, img_height = img.size
        except Exception as e:
            print(f"이미지 파일을 열 수 없습니다: {img_path}. 오류: {e}")
            continue
        
        # 이미지 파일명에 비디오 ID 추가
        new_img_filename = f"{video_id}_{img_filename}"
        img_output_path = os.path.join(output_img_dir, new_img_filename)
        shutil.copyfile(img_path, img_output_path)
        
        # 주석 파일명에 비디오 ID 추가
        txt_filename = f"{video_id}_frame_{frame_number}_timestamp_{timestamp}.txt"
        txt_path = os.path.join(output_label_dir, txt_filename)
        
        # YOLO 형식의 주석 파일 생성
        with open(txt_path, 'w') as f:
            bbox = annotation['bounding_box']
            x_center, y_center, width, height = convert_bbox(img_width, img_height, bbox)
            
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
            
            keypoints = annotation['keypoints']
            for i in range(1, 16):
                kp = keypoints.get(str(i))
                if kp is not None:
                    x, y = convert_keypoint(img_width, img_height, kp)
                    f.write(f" {x:.6f} {y:.6f} 2")  # 2는 가시성을 나타냄 (보이는 키포인트)
                else:
                    f.write(" 0 0 0")  # 0 0 0은 키포인트가 없음을 나타냄
            
            f.write("\n")
        
        print(f"변환 완료: {txt_path}, {img_output_path}")
    
    print(f"메타데이터 저장 완료: {meta_path}")
    
    if missing_images:
        print("\n누락된 이미지 파일:")
        for img in missing_images:
            print(img)
        print(f"\n총 {len(missing_images)}개의 이미지 파일이 누락되었습니다.")


def process_all_json_files(base_json_folder, base_img_folder, output_img_folder, output_label_folder, output_meta_folder):
    for animal, pose_classes in [('CAT', CAT_POSE_CLASSES), ('DOG', DOG_POSE_CLASSES)]:
        json_folder = os.path.join(base_json_folder, animal)
        img_folder = os.path.join(base_img_folder, animal)
        
        for pose in os.listdir(json_folder):
            json_pose_folder = os.path.join(json_folder, pose)
            img_pose_folder = os.path.join(img_folder, pose)
            
            if not os.path.isdir(json_pose_folder):
                continue
            
            class_id = pose_classes.get(f"{animal}_{pose}", -1)
            if class_id == -1:
                print(f"알 수 없는 pose: {pose}")
                continue
            
            for json_file in os.listdir(json_pose_folder):
                if json_file.endswith('.json'):
                    json_file_path = os.path.join(json_pose_folder, json_file)
                    img_subfolder = os.path.splitext(json_file)[0]
                    img_folder_path = os.path.join(img_pose_folder, img_subfolder)
                    
                    convert_json_to_yolo(json_file_path, img_folder_path, output_img_folder, output_label_folder, output_meta_folder, class_id)
    
    print("모든 파일이 처리되었습니다.")

def create_yaml_file(base_output_folder, yaml_file_path):
    data = {
        'train': os.path.join(base_output_folder, 'images', 'train'),
        'val': os.path.join(base_output_folder, 'images', 'val'),
        'nc': len(ALL_POSE_CLASSES),
        'names': list(ALL_POSE_CLASSES.keys())
    }
    
    with open(yaml_file_path, 'w') as f:
        yaml.dump(data, f, default_flow_style=False)
    
    print(f"YAML 파일이 생성되었습니다: {yaml_file_path}")

# 메인 실행 코드
base_output_folder = r"E:\pet_data\yolo_dataset\test"

# Training 데이터 처리
base_json_folder = r"E:\pet_data\Training"
base_img_folder = r"E:\pet_data\Training\img"
output_img_folder = os.path.join(base_output_folder, 'images')
output_label_folder = os.path.join(base_output_folder, 'labels')
output_meta_folder = os.path.join(base_output_folder, 'metadata')

# 출력 폴더 생성
os.makedirs(output_img_folder, exist_ok=True)
os.makedirs(output_label_folder, exist_ok=True)
os.makedirs(output_meta_folder, exist_ok=True)

process_all_json_files(base_json_folder, base_img_folder, output_img_folder, output_label_folder, output_meta_folder)

# YAML 파일 생성
yaml_file_path = os.path.join(base_output_folder, 'data.yaml')
create_yaml_file(base_output_folder, yaml_file_path)

print("처리가 완료되었습니다.")

알 수 없는 pose: ARCH
알 수 없는 pose: ARMSTRETCH
알 수 없는 pose: FOOTPUSH
알 수 없는 pose: GETDOWN
알 수 없는 pose: GROOMING
알 수 없는 pose: HEADING
알 수 없는 pose: img
알 수 없는 pose: LAYDOWN
알 수 없는 pose: LYING
알 수 없는 pose: ROLL
알 수 없는 pose: SITDOWN
알 수 없는 pose: TAILING
알 수 없는 pose: WALKRUN
알 수 없는 pose: BODYLOWER
알 수 없는 pose: BODYSCRATCH
알 수 없는 pose: BODYSHAKE
알 수 없는 pose: FEETUP
알 수 없는 pose: FOOTUP
알 수 없는 pose: HEADING
알 수 없는 pose: LYING
알 수 없는 pose: MOUNTING
알 수 없는 pose: SIT
알 수 없는 pose: TAILING
알 수 없는 pose: TAILLOW
알 수 없는 pose: TURN
알 수 없는 pose: WALKRUN
알 수 없는 pose: ARCH
알 수 없는 pose: ARMSTRETCH
알 수 없는 pose: FOOTPUSH
알 수 없는 pose: GETDOWN
알 수 없는 pose: GROOMING
알 수 없는 pose: HEADING
알 수 없는 pose: LAYDOWN
알 수 없는 pose: LYING
알 수 없는 pose: ROLL
알 수 없는 pose: SITDOWN
알 수 없는 pose: TAILING
알 수 없는 pose: WALKRUN
알 수 없는 pose: BODYLOWER
알 수 없는 pose: BODYSCRATCH
알 수 없는 pose: BODYSHAKE
알 수 없는 pose: FEETUP
알 수 없는 pose: FOOTUP
알 수 없는 pose: HEADING
알 수 없는 pose: LYING
알 수 없는 pose: MOUNTING
알 수 없는 pose: SIT
알 수 없는 pose: TAILING
알 수